https://www.evidentlyai.com/ranking-metrics/evaluating-recommender-systems

# set up

In [1]:
from qdrant_client import QdrantClient

client = QdrantClient(host="localhost", port=6333)          # or: client = QdrantClient(url="http://localhost:6333")

In [2]:
try:
    response = client.get_collections()
    print("Successfully connected to Qdrant.")
    print("Available collections:", response, sep="\n")
except Exception as e:
    print("Failed to connect to Qdrant:", e)

Successfully connected to Qdrant.
Available collections:
collections=[CollectionDescription(name='corpus_halong-trained')]


In [3]:
import os
import pandas as pd
import numpy as np
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.schema import TextNode

# evaluation function

In [4]:
def compute_MRR(retriever, test_set):
    mrr = 0
    for _, row in test_set.iterrows():
        retrieved_nodes = retriever.retrieve(row['question'])
        for j, node in enumerate(retrieved_nodes):
            if node.text == row['context']:
                mrr += 1/(j+1)
                break
    return mrr/len(test_set)

In [5]:
def compute_hit_rate(retriever, test_set):
    hit_rate = 0
    for _, row in test_set.iterrows():
        retrieved_nodes = retriever.retrieve(row['question'])
        for j, node in enumerate(retrieved_nodes):
            if node.text == row['context']:
                hit_rate += 1
                break
    return hit_rate/len(test_set)

In [6]:
def compute_ndcg(retriever, test_set):
    ndcg_values = []
    
    for _, row in test_set.iterrows():
        retrieved_nodes = retriever.retrieve(row['question'])
        relevance_scores = [1 if node.text == row['context'] else 0 for node in retrieved_nodes]

        if max(relevance_scores) == 0:  # No relevant document retrieved
            ndcg_values.append(0)
            continue

        # Compute DCG
        dcg = sum(rel / np.log2(i + 2) for i, rel in enumerate(relevance_scores))

        # Compute IDCG (Ideal DCG) using sorted relevance scores
        ideal_relevance_scores = sorted(relevance_scores, reverse=True)
        idcg = sum(rel / np.log2(i + 2) for i, rel in enumerate(ideal_relevance_scores))

        # Compute NDCG
        ndcg = dcg / idcg if idcg > 0 else 0
        ndcg_values.append(ndcg)

    return sum(ndcg_values)/len(test_set)


# No word segmentation

In [4]:
corpus = pd.read_csv('data-processed/corpus.csv')
corpus.head(3)

,context,article,document
0,Điều 9. Tuyển bổ sung và loại ra khỏi chương t...,Điều 9. Tuyển bổ sung và loại ra khỏi chương t...,QUY ĐỊNH ĐÀO TẠO CHƯƠNG TRÌNH TÀI NĂNG
1,Điều 4. Kiểm tra xếp lớp đầu khóa cho sinh viê...,Điều 4. Kiểm tra xếp lớp đầu khóa cho sinh viê...,QUY ĐỊNH ĐÀO TẠO NGOẠI NGỮ ĐỐI VỚI HỆ ĐẠI HỌC ...
2,Điều 5. Chương trình đào tạo CT CLC được xây d...,Điều 5. Chương trình đào tạo,QUY ĐỊNH ĐÀO TẠO CHƯƠNG TRÌNH CHẤT LƯỢNG CAO


In [5]:
nodes = [
    TextNode(
        text=row['context'],
        metadata={
            "article": row['article'],
            "document": row['document'],
        } 
    )
    for _, row in corpus.iterrows()
]

In [6]:
corpus_question_test_map = pd.read_csv("data-processed/corpus-question-test-map.csv")
corpus_question_test_map.head(3)

,context,article,document,question
0,"Điều 8. Xây dựng, thẩm định học liệu điện tử 1...","Điều 8. Xây dựng, thẩm định học liệu điện tử",QUY ĐỊNH DẠY VÀ HỌC THEO PHƯƠNG THỨC TRỰC TUYẾ...,Học liệu điện tử sau khi được thông qua bởi ĐV...
1,"Điều 20. Hồ sơ, trình tự, thủ tục chỉnh sửa nộ...","Điều 20. Hồ sơ, trình tự, thủ tục chỉnh sửa nộ...","QUY CHẾ Văn bằng, chứng chỉ của Trường Đại học...",Có thể nộp bản sao giấy khai sinh cho hồ sơ đề...
2,Điều 23. Điểm Miễn 1. Điểm BL - Sinh viên đã t...,Điều 23. Điểm Miễn,QUY CHẾ ĐÀO TẠO THEO HỌC CHẾ TÍN CHỈ CHO HỆ ĐẠ...,Quy định về điểm M trong trường hợp sinh viên ...


## halong trained

In [7]:
embed_model_halong_trained = HuggingFaceEmbedding(model_name="KhoaUIT/Halong-UIT-R2GQA", max_length=512)

In [8]:
# QdrantVectorStore: docs.llamaindex.ai/en/stable/api_reference/storage/vector_store/qdrant/
qdrant_vector_store = QdrantVectorStore(client=client,
                                        collection_name="corpus_halong-trained",
                                        enable_hybrid=True)

storage_context = StorageContext.from_defaults(vector_store=qdrant_vector_store)

In [10]:
# create index for the first time

index_halong_trained = VectorStoreIndex(
    nodes,
    storage_context=storage_context,
    embed_model=embed_model_halong_trained,
)

Collection corpus_halong-trained already exists, skipping collection creation.


In [ ]:
# Load the index from the existing vector store

index_halong_trained = VectorStoreIndex.from_vector_store(
    vector_store=qdrant_vector_store,
    storage_context=storage_context,
    embed_model=embed_model_halong_trained
)

In [15]:
retriever_halong_trained  = index_halong_trained.as_retriever(
    similarity_top_k=10,
    vector_store_query_mode="hybrid",
    alpha=0.5,
)

In [16]:
compute_hit_rate(retriever_halong_trained, corpus_question_test_map)

0.9661885245901639

In [17]:
compute_MRR(retriever_halong_trained, corpus_question_test_map)

0.755805604345564

In [20]:
compute_ndcg(retriever_halong_trained, corpus_question_test_map)

0.808035384892791